## AIMS
The aims of this script are to create events files that are compatible with bids for the RED Go/NoGo task

What we need are from the events.txt files (trials.txt have much more info but this one has the onset times we need to specify)

Required:
- onset
- duration

What we should also specify
- condition (how we have it named in the actual file)
- trial_type (go/nogo)
- emotion

In [28]:
#import modules
import os
import re
import csv, operator
import pandas as pd
import scipy as sio
import numpy as np

In [1]:
# First identify directories
files_dir ='/imaging/su01/RED/GNG Task Data'
output_dir = '/imaging/su01/RED/bids'

# Identify subject list (red_files correspond to bids_ids order)
red_events = ["99003_events.txt", "99004_events.txt",  "99005_events.txt", "99007_events.txt",
           "99009_events.txt", "99010_events.txt", "99011_events.txt", "99012_events.txt", 
           "99013_events.txt", "99014_events.txt", "99016_events.txt", "99018_events.txt", 
           "99019_events.txt", "99020_events.txt", "99021_events.txt", "99023_events.txt", 
           "99025_events.txt", "99026_events.txt", "99027_events.txt", "99028_events.txt", 
           "99030_events.txt", "99031_events.txt", "99034_events.txt", "99035_events.txt", 
           "99037_events.txt", "99038_events.txt", "99040_events.txt", "99044_events.txt", 
           "99047_events.txt", "99048_events.txt", "99051_events.txt", "99053_events.txt", 
           "99057_events.txt", "99058_events.txt", "99059_events.txt", "99063_events.txt", 
           "99064_events.txt", "99068_events.txt", "99070_events.txt", "99071_events.txt", 
           "99072_events.txt", "99073_events.txt", "99076_events.txt", "99078_events.txt", 
           "99080_events.txt", "99083_events.txt", "99086_events.txt", "99087_events.txt", 
           "99089_events.txt", "99090_events.txt", "99091_events.txt", "99092_events.txt", 
           "99094_events.txt", "99096_events.txt", "99097_events.txt", "99099_events.txt", 
           "99101_events.txt", "99102_events.txt", "99103_events.txt", "99106_events.txt", 
           "99107_events.txt", "99111_events.txt", "99112_events.txt", "99114_events.txt", 
           "99115_events.txt", "99118_events.txt", "99119_events.txt", "99121_events.txt", 
           "99124_events.txt", "99125_events.txt", "99129_events.txt", "99134_events.txt", 
           "99141_events.txt", "99142_events.txt", "99139_events.txt"]

red_files = ["99003_trials.txt", "99004_trials.txt", "99005_trials.txt", "99007_trials.txt",
           "99009_trials.txt", "99010_trials.txt", "99011_trials.txt", "99012_trials.txt", 
           "99013_trials.txt", "99014_trials.txt", "99016_trials.txt", "99018_trials.txt", 
           "99019_trials.txt", "99020_trials.txt", "99021_trials.txt", "99023_trials.txt", 
           "99025_trials.txt", "99026_trials.txt", "99027_trials.txt", "99028_trials.txt", 
           "99030_trials.txt", "99031_trials.txt", "99034_trials.txt", "99035_trials.txt", 
           "99037_trials.txt", "99038_trials.txt", "99040_trials.txt", "99044_trials.txt", 
           "99047_trials.txt", "99048_trials.txt", "99051_trials.txt", "99053_trials.txt", 
           "99057_trials.txt", "99058_trials.txt", "99059_trials.txt", "99063_trials.txt", 
           "99064_trials.txt", "99068_trials.txt", "99070_trials.txt", "99071_trials.txt", 
           "99072_trials.txt", "99073_trials.txt", "99076_trials.txt", "99078_trials.txt", 
           "99080_trials.txt", "99083_trials.txt", "99086_trials.txt", "99087_trials.txt", 
           "99089_trials.txt", "99090_trials.txt", "99091_trials.txt", "99092_trials.txt", 
           "99094_trials.txt", "99096_trials.txt", "99097_trials.txt", "99099_trials.txt", 
           "99101_trials.txt", "99102_trials.txt", "99103_trials.txt", "99106_trials.txt", 
           "99107_trials.txt", "99111_trials.txt", "99112_trials.txt", "99114_trials.txt", 
           "99115_trials.txt", "99118_trials.txt", "99119_trials.txt", "99121_trials.txt", 
           "99124_trials.txt", "99125_trials.txt", "99129_trials.txt", "99134_trials.txt", 
           "99141_trials.txt", "99142_trials.txt", "99139_trials.txt"]


bids_ids = ["sub-002", "sub-003", "sub-001", "sub-004", "sub-009", "sub-010", "sub-006", "sub-016", 
            "sub-005", "sub-008", "sub-012", "sub-011", "sub-007", "sub-014", "sub-013", "sub-019",
            "sub-017", "sub-015", "sub-021", "sub-020", "sub-022", "sub-018", "sub-023", "sub-026",
            "sub-028", "sub-025", "sub-027", "sub-042", "sub-024", "sub-050", "sub-051", "sub-068",
            "sub-032", "sub-031", "sub-030", "sub-074", "sub-037", "sub-029", "sub-035", "sub-036",
            "sub-040", "sub-041", "sub-034", "sub-038", "sub-039", "sub-047", "sub-043", "sub-046",
            "sub-044", "sub-048", "sub-045", "sub-049", "sub-063", "sub-069", "sub-056", "sub-054",
            "sub-058", "sub-052", "sub-057", "sub-053", "sub-070", "sub-064", "sub-055", "sub-062",
            "sub-065", "sub-061", "sub-073", "sub-066", "sub-059", "sub-060", "sub-072", "sub-075",
            "sub-077", "sub-076", "sub-078"]

In [4]:
#check current working directory
import os
os.getcwd()

'/Users/stepheniuh/PycharmProjects/fmriprep_prc'

In [5]:
# Scratchpad
import pandas as pd

events = pd.read_table("~/ownCloud/PhD/RED/Go-NoGo data/GNG Task Data/99013_events.txt", sep='\t')
print(events)

events_df = pd.DataFrame(data=events)
events_df

              time              event
0     5.848265e+04        run 0 onset
1     1.189224e+06        buttonpress
2     5.848265e+04  block 0 onset c_n
3     1.204328e+06        buttonpress
4     1.204398e+06          MRI pulse
...            ...                ...
1162  2.283479e+06       button press
1163  2.284363e+06     trial 17 onset
1164  2.285780e+06         stim onset
1165  2.286063e+06       signal onset
1166  2.286587e+06       button press

[1167 rows x 2 columns]


,time,event
0,5.848265e+04,run 0 onset
1,1.189224e+06,buttonpress
2,5.848265e+04,block 0 onset c_n
3,1.204328e+06,buttonpress
4,1.204398e+06,MRI pulse
...,...,...
1162,2.283479e+06,button press
1163,2.284363e+06,trial 17 onset
1164,2.285780e+06,stim onset
1165,2.286063e+06,signal onset


In [6]:
# just check what kind of variables these are
test_df = events_df
test_df

test_df.dtypes

time     float64
event     object
dtype: object

In [18]:
## Get first MRI pulse value

# Grab all the rows with "MRI pulse" in the event column
pulse = test_df[test_df.event.str.contains('MRI pulse')]

# Extract first column into new dataframe (this only has the time)
pulse_value = pulse.time

# Make this into a list to then save into a variable value to call
pulse_value = pulse_value.values.tolist()
mri_pulse = pulse_value[0] #use this as subtraction value

In [105]:
## Identify all conditions and blocks

# Extract rows containing block names
block_names = ['s_n', 's_g', 'n_n', 'n_g', 'h_n', 'h_g', 'c_n', 'c_g']

blocks = test_df[test_df.event.str.contains('|'.join(block_names ))]
sLength = len(blocks['event'])
blocks = blocks.assign(emotion=pd.Series(block_names))
blocks
# blocks = blocks.assign(emotion=pd.Series(np.random.randn(sLength)).values)
# for i in block_names:
#     if i in blocks['event']:
#         blocks['emotion'] = i
#         print(i)

# for i in block_names:
#     if np.where(blocks)
#     print(str(i))

,time,event,emotion
2,5.848265e+04,block 0 onset c_n,n_n
71,1.549647e+09,block 1 onset h_n,NaN
141,1.549647e+09,block 2 onset s_g,NaN
216,1.549647e+09,block 3 onset s_n,NaN
286,1.549647e+09,block 4 onset h_g,NaN
360,1.549647e+09,block 5 onset n_g,NaN
435,1.549647e+09,block 6 onset n_n,NaN
506,1.549647e+09,block 7 onset c_g,NaN
583,1.736830e+06,block 0 onset n_g,NaN
658,1.549647e+09,block 1 onset c_n,NaN


In [98]:
emotion = []
for i in block_names:
    if i in blocks['event']:
        emotion.append(block_names[i])

In [99]:
for emo in blocks['event']:
    if emo in block_names[]:
        emotion.append(block_names[emo])

[]

In [16]:
d = {'conditions': ['s_n', 's_g', 'n_n', 'n_g', 'h_n', 'h_g', 'c_n', 'c_g'],
     'onsets': [0, 0, 0, 0, 0, 0, 0, 0],
     'duration': [60, 60, 60, 60, 60, 60, 60, 60],
     'trial_type': ['nogo', 'go', 'nogo', 'go', 'nogo', 'go', 'nogo', 'go'],
    'emotion': ['sad', 'sad', 'neutral', 'neutral', 'happy', 'happy', 'control', 'control']}
df = pd.DataFrame(data=d)
df

,conditions,onsets,duration,trial_type,emotion
0,s_n,0,60,nogo,sad
1,s_g,0,60,go,sad
2,n_n,0,60,nogo,neutral
3,n_g,0,60,go,neutral
4,h_n,0,60,nogo,happy
5,h_g,0,60,go,happy
6,c_n,0,60,nogo,control
7,c_g,0,60,go,control


In [13]:
df.iloc[1, 1]
events_df.iloc[0,1]

'run 0 onset'

In [32]:
# define subtraction from MRI pulse
def onset_time(file, onset_subtraction):
    with open(file, 'r') as infile:
        reader = csv.DictReader(infile, delimiter='\t', skipinitialspace=True)
        df = pd.DataFrame(data=reader)
        pulse = reader.iloc[4,0] #this is the first MRI pulse in each text file
        condition = search
        for row in reader:
            val = row['event']
            try:
                f = float(val) #this is to discard NAs and not count them
                sum += f
                count += 1
            except:
                continue

        fdmap[file] = sum / count if count  != 0 else 0

TypeError: unhashable type: 'list'